In [1]:
import pandas as pd

In [2]:
df_q2 = pd.read_csv('capital_bike_data/2017_capitalbike_data/2017Q2-capitalbikeshare-tripdata.csv')
df_q1 = pd.read_csv('capital_bike_data/2017_capitalbike_data/2017Q1-capitalbikeshare-tripdata.csv')
df_q1.shape

(646510, 9)

In [3]:
df_q4 = pd.read_csv('capital_bike_data/2017_capitalbike_data/2017Q4-capitalbikeshare-tripdata.csv')
df_q3 = pd.read_csv('capital_bike_data/2017_capitalbike_data/2017Q3-capitalbikeshare-tripdata.csv')

In [4]:
df_q3.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,2762,2017-07-01 00:01:09,2017-07-01 00:47:11,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,W21474,Casual
1,2763,2017-07-01 00:01:24,2017-07-01 00:47:27,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,W22042,Casual
2,690,2017-07-01 00:01:45,2017-07-01 00:13:16,31122,16th & Irving St NW,31299,Connecticut Ave & R St NW,W01182,Member
3,134,2017-07-01 00:01:46,2017-07-01 00:04:00,31201,15th & P St NW,31267,17th St & Massachusetts Ave NW,W22829,Member
4,587,2017-07-01 00:02:05,2017-07-01 00:11:52,31099,Madison & N Henry St,31907,Franklin & S Washington St,W22223,Casual


In [40]:
df_con = [df_q1, df_q2, df_q3, df_q4]
df_2017 = pd.concat(df_con)
df_2017.shape

(3757777, 9)

In [41]:
import requests
response = requests.get('https://gbfs.capitalbikeshare.com/gbfs/en/station_information.json')
stations = pd.DataFrame(response.json()['data']['stations'])
stations.drop(['eightd_station_services', 'external_id', 'rental_url'], axis=1, inplace=True)
stations.head()

,capacity,eightd_has_key_dispenser,electric_bike_surcharge_waiver,has_kiosk,lat,lon,name,region_id,rental_methods,short_name,station_id
0,19,True,False,True,38.857405,-77.051132,Crystal City Metro / 18th St & S Bell St,41,"[CREDITCARD, KEY]",31007,8
1,19,False,False,True,38.905340,-77.046774,21st & M St NW,42,"[CREDITCARD, KEY]",31212,53
2,19,False,False,True,38.902221,-77.059219,Georgetown Harbor / 30th St NW,42,"[CREDITCARD, KEY]",31215,72
3,23,False,False,True,38.900283,-77.029822,13th St & New York Ave NW,42,"[CREDITCARD, KEY]",31227,91
4,35,False,False,True,38.902061,-77.038322,17th & K St NW / Farragut Square,42,"[CREDITCARD, KEY]",31233,106


In [42]:
# merge_asof
# Perform an asof merge. This is similar to a left-join except that we match on nearest key rather than equal keys.

In [43]:
# check if random station name of merged-quarterly-df is in station df 
stations['short_name'].where(stations['short_name'] == str('31099')).count()

1

In [44]:
# Ok, cool! Eine Übereinstimmung.
# ACHTUNG: station short_name is float in station df / string in merged df
# make it all int

In [45]:
stations['short_name'] = stations['short_name'].astype(int)
df_2017['Start station number'] = df_2017['Start station number'].astype(int)
df_2017['End station number'] = df_2017['End station number'].astype(int)
df_2017['Start station number'].where(df_2017['Start station number'] == 31099).count()

1927

In [46]:
df_grouped=df_2017.groupby(['Start station number', 'End station number'])[['Bike number']].count().reset_index()

In [47]:
df_grouped.head()

,Start station number,End station number,Bike number
0,31000,31000,268
1,31000,31001,17
2,31000,31002,68
3,31000,31003,82
4,31000,31004,22


In [48]:
stat_location = stations[['lat', 'lon', 'short_name']]

In [49]:
df_grouped = pd.merge_asof(left=df_grouped.sort_values(by=['Start station number']), right=stat_location.sort_values(by=['short_name']), left_on='Start station number', right_on='short_name')
df_grouped['lat_start'] = df_grouped['lat']
df_grouped['lon_start'] = df_grouped['lon']
df_grouped.drop(['lon', 'lat'], axis=1, inplace=True)
#df_2017_viz = df_2017_stations['Start station number', 'End station number', 'lat_start', 'lon_start']
df_grouped.head()

,Start station number,End station number,Bike number,short_name,lat_start,lon_start
0,31000,31000,268,31000,38.858971,-77.05323
1,31000,31271,3,31000,38.858971,-77.05323
2,31000,31269,1,31000,38.858971,-77.05323
3,31000,31265,1,31000,38.858971,-77.05323
4,31000,31264,2,31000,38.858971,-77.05323


In [50]:
df_grouped = pd.merge_asof(left=df_grouped.sort_values(by=['End station number']), right=stat_location.sort_values(by=['short_name']), left_on='End station number', right_on='short_name')
df_grouped['lat_end'] = df_grouped['lat']
df_grouped['lon_end'] = df_grouped['lon']
df_grouped['bike_count'] =df_grouped['Bike number']
df_grouped.drop(['lon', 'lat', 'short_name_x', 'short_name_y', 'Bike number'], axis=1, inplace=True)
#df_grouped.to_csv('bike_stations_location_dc.csv')
df_grouped.head()

,Start station number,End station number,lat_start,lon_start,lat_end,lon_end,bike_count
0,31000,31000,38.858971,-77.053230,38.858971,-77.05323,268
1,31066,31000,38.867262,-77.072315,38.858971,-77.05323,8
2,31219,31000,38.893028,-77.026013,38.858971,-77.05323,4
3,31913,31000,38.820058,-77.062821,38.858971,-77.05323,13
4,31067,31000,38.862478,-77.086599,38.858971,-77.05323,1


### Step 2: Input weather data from Weather API we collected

In [51]:
#weather = pd.read_csv('washington_dc_weather/allweather_2017.csv')
#weather['datetime'] = pd.to_datetime(weather['time'], unit='s')
# JupiterLab died, RAM overflow (?) --> Step 2 unimplemented

### Step 3: Visualisation

In [52]:
from gcmap import GCMapper, Gradient
from sklearn.preprocessing import MinMaxScaler

In [53]:
#df_grouped['lat_end'][0].reshape(1,-1)

tried scaling:

mms_los = MinMaxScaler()
mms_las = MinMaxScaler()
mms_loe = MinMaxScaler()
mms_lae = MinMaxScaler()

df_grouped['lon_start'] = mms_los.fit((df_grouped['lon_start']).reshape(1, -1))
df_grouped['lat_start'] = mms_las.fit((df_grouped['lat_start']).reshape(1, -1))
df_grouped['lon_end'] = mms_loe.fit((df_grouped['lon_end']).reshape(1, -1))
df_grouped['lat_end'] = mms_lae.fit((df_grouped['lat_end']).reshape(1, -1))

df_grouped.head()

In [56]:
# create gradient to color the routes according to the number of bikes
grad = Gradient(((0, 0, 0, 0), (0.5, 204, 0, 153), (1, 255, 204, 230)))
# initialize GCMapper and set data
x = 0
gcm = GCMapper(cols=grad, height=1080, width=1920, gc_resolution=100, proj='eqc')
gcm.set_data(df_grouped['lon_start']+x, df_grouped['lat_start']+x, df_grouped['lon_end']+x,
             df_grouped['lat_end']+x, df_grouped['bike_count'])
# run & save
img = gcm.draw()
img.save('bike_map_gcmap.png')

projection technics:

- 'lcc +lat_1=48 +lat_2=33 +lon_0=-100 +ellps=WGS84'
- 'rcc'
- 'eqc'
- 'mill
- Another option is the Miller projection [+proj=mill] or the Equidistant Cylindrical (Plate Carrée) [+proj=eqc] if you're wanting to keep the rectangular shape. (I have not experimented with rotating the perspective - lambda, phi, gamma - on these projections though)

In [57]:
import plotly

In [ ]:
plot.geo()